# Introduction to Computational Geometry

A subset $S$ of the plane is called _convex_ if and only if for any pair of points $p, q \in S$, the line segment $\overline{pq}$ is completely contained in $S$. The convex hull _CH(S)_ of a set $S$ is the smallest convex set that contains $S$. To be more precise, it is the intersection of all convex sets that contain $S$.

**Note**: This means that from the finite set of points $P$ from $S$ we want to find a polygon $P$ that contains _all_ points from $S$ such that the polygon is the $convex hull$ of $S$. However, it is good to note that convex hull _doesn't_ always have to be a polygon.

We will study the problem of computing the convex hull of finite set $P$ of $n$ points in the plane.

## Convex Hull problem
The convex hull of $P$ is a convex polygon. A natural way to represent a polygon is by listing its vertices in clockwise order, starting with an arbitrary one. So the problem we want to solve is: given a set $P = \{p_1, p_2, \dots, p_n\}$ of points in the plane, compute a list that contains those points from $P$ that are the vertices of $CH(P)$ listed in clockwise order.

The intersection of all convex sets that contain these points from $P$ doesn't help us finding the convex hull of $P$, since for computing $CH(P)$ this way, we have to consider infinitely many convex sets and intersect them. What we want to see are the edges of $CH(P)$. 

Both endpoints $p$ and $q$ of such an edge are points of $P$ and if we direct the line through $p$ and $q$ such that $CH(P)$ lies on the right, then all the points of $P$ must lie right of this line. The reverse is also true: if all points of $P \backslash \{p, q\}$ lie on the right of the directed line through $p$ and $q$, then $\overline{pq}$ is an edge of $CH(P)$.

### Slow Convex Hull Algorithm
Using this understanding, we can develop a naive algorithm to compute a convex hull

**Algorithm** SlowConvexHull($P$) \
_Input_ A set $P$ of points in the plane.\
_Output_ A list $L$ containing the vertices of $CH(P)$ in clockwise order.
1. $E \leftarrow \emptyset$
2. **for** all ordered pairs $(p, q) \in P \times P$ with $p$ not equal to $q$
3. &emsp; **do** _valid_ $\leftarrow$ **true**
4. &emsp;&emsp; **for** all points $r \in P$ not equal to $p$ or $q$
5. &emsp;&emsp;&emsp; **do** if r _lies_ to the left of the directed line from $p$ to $q$
6. &emsp;&emsp;&emsp;&emsp; **then** _valid_ $\leftarrow$ false
7. &emsp;&emsp; **if** _valid_ **then** Add the directed edge $\vec{pq}$ to $E$.
8. From the set $E$ of edges construct a list $L$ of vertices $CH(P)$, sorted in clockwise order

For analyzing the time complexity of *SlowConvexHull*, we check $n^2 - n$ pairs of point. For each pair, we look at the $n - 2$ other points to see whether they all lie on the right side. This will take $O(n^3)$ time in total. The final step takes $O(n^2)$ time in total, so the total running time is $O(n^3)$.

### Slow Convex Hull Algorithm - Degenerate cases and robustness

Although this could be a good way to think about the convex hull, it is still naive and not very efficient, since the cubic running time algorithms takes its toll pretty quickly for larger set of points, so it is not quite practical for many applications.

After a quick observation, we can see there are some potential issues that we haven't considered. When deriving the criterion of when a pair $p, q$ defines an edge of $CH(P)$, a point $r$ does not always lie to the right or to the left of the line through $p$ and $q$, it can also happen that it lies $on$ this line. This is what we call _degenerate case_, or _degeneracy_. To make the algorithm correct in the presence of degeneracies, we must reformulate the criterion above as follows: a directed edge $\vec{pq}$ is an edge of $CH(P)$ if and only if all other points $r \in P$ lie either strictly to the right of the directed line through $p$ and $q$, or they lie on the open line segment $\overline{pq}$.

We also have an issue we have ignored while developing an algorithm that can influence the correctness of the result of our algorithm. We implicitly assumed that we can, somehow, test exactly whether a point lies to the right or to the left of a given line. This is not necessarily true: if the points are given in floating point coordinates and the computations are done using floating point arithmetic, then there will be rounding errors that may distort the outcome of the tests. If it happens that three points $p, q, r \in P$ are nearly collinear, the floating point arithmetic could produce two results because of the rounding errors: either the algorithm will accept all three edges or none of them and we could get a convex hull from this algorithm that instead of producing exactly one edge from these points, we get two or none of them because of the rounding errors.

Although the algorithm is correct and is capable to handle special cases, it is not _robust_; small errors in the computations can make it fail in completely unexpected ways. The problem is that we have proven the correctness assuming that we can compute exactly with real numbers.

### Incremental Algorithm for computing Convex Hull (Graham's scan)

We have designed the first geometric algorithm, but it is slow, it solves degenerate cases in pretty awkward way and it is not robust. We will now try to solve the Convex Hull problem with a better solution.

We will develop an _incremental algorithm_. This means that we will add the points in $P$ one by one, updating our solution after each addition. We will add the points from left to right.

First, we sort the points by $x$-coordinate, obtaining a sorted sequence $p_1, p_2, \dots, p_n$ and then we will add them in that order. Because we are working from left to right, it would be convenient if the convex hull vertices were also ordered from left to right as they occur along the boundary. But this is not the case, therefore we first compute only those convex hull vertices that lie on the *upper hull*, which is the part of the convex hull running from the leftmost point $p_1$ to the rightmost point $p_n$ when the vertices are listed in clockwise order. In other words, the upper hull contains the convex hull edges bounding the convex hull from above. In a second scan, which is performed right to left, we compute the remaining part of the convex hull, the _lower hull_.

The basic step in the incremental algorithm is the update of the upper hull after adding a point $p_i$. In other words, given the upper hull of the points $p_1, \dots, p_{i-1}$, we have to compute the upper hull of $p_1, \dots, p_i$. When we walk around the boundary of a polygon in clockwise order, we make a turn at every vertex. For an arbitrary polygon this can be both a right turn and a left turn, but for a convex polygon every turn must be a right turn. This suggests handling the addition of $p_i$ in the following way: Let $L_{upper}$ be a list that stores the upper vertices in left-to-right order. We first append $p_i$ to $L_{upper}$. This is correct because $p_i$ is the rightmost point of the ones added so far, so it must be on the upper hull. Next, we check whether the last three points in $L_{upper}$ make a right turn. If this is the case there is nothing more to do; $L_{upper}$ contains the vertices of the upper hull of $p_1, \dots, p_i$ and we can proceed to the next point, $p_{i+1}$. But if the last three points make a left turn, we have to delete the middle one from the upper hull. In this case we are not finished yet: it could be that the new last three points still do not make a right turn, in which case we again have to delete the middle one. We continue this manner until the last three points make a right turn or until there are only two points left.

We now present the algorithm, which computer upper hull and lower hull, which computes from right to left analogous to the process of computing upper hull.

**Algorithm** ConvexHull($P$) \
_Input_ A set $P$ of points in the plane.\
_Output_ A list $L$ containing the vertices of $CH(P)$ in clockwise order.

1. Sort the points by $x$-coordinate, resulting in a sequence $p_1, p_2, \dots, p_n$
2. Put the points $p_1$ and $p_2$ in a list $L_{upper}$ with $p_1$ as the first point
3. **for** $i$ $\leftarrow$ 3 **to** $n$
4. &emsp; **do** Append $p_i$ to $L_{upper}$.
5. &emsp;&emsp; **while** $L_{upper}$ contains more than two points **and** the last three points in $L_{upper}$ do not make a right turn
6. &emsp;&emsp;&emsp; **do** Delete the middle of the last three points from $L_{upper}$
7. Put points $p_n$ and $p_{n-1}$ in a list $L_{lower}$ with $p_n$ as the first point.
8. **for** $i \leftarrow n - 2$ **downto** 1
9. &emsp; **do** Append $p_i$ to $L_{lower}$
10. &emsp;&emsp; **while** $L_{lower}$ contains more than 2 points **and** the last three points in $L_{lower}$ do not make a right turn
11. &emsp;&emsp;&emsp; **do** Delete the middle of the last three points from $L_{lower}$.
12. Remove the first and the last point from $L_{lower}$ to avoid duplication of the points where the upper and lower hull meet.
13. Append $L_{lower}$ to $L_{upper}$ and call the resulting list $L$
14. **return** $L$

### Incremental Convex Hull algorithm - degenerate cases and robustness

Once again, when we look closer, we realize that the above algorithm is not correct. Without mentioning it, we made the assumption that no two points have the same $x$-coordinate. If this assumption is not valid, the order on $x$-coordinate is not well defined. We have to generalize the ordering in a suitable way. Rather than using only the $x$-coordinate of the points to define the order, we use lexicographic order. This means that we first sort by $x$-coordinate and if points have the same $x$-coordinate, we sort them by $y$-coordinate.

Another special case we have ignored is that three points for which we have to determine whether they make left or right turn lie on a straight line. In this case, the middle point should not occur on the convex hull, so collinear points must be treated as if they make the left turn. 

With these modifications, the algorithm correctly computes the convex hull, where the first scan computes the upper hull, which is now defined  as the part of the convex hull running from the lexicographically smallest vertex to the lexicographically largest vertex, and the second scan computes the remaining part of the convex hull.

When rounding error occurs, it can happen that a point is removed from the convex hull although it should be there or that a point inside the real convex hull is not removed. But, the structural integrity of the algorithm is unharmed; it will compute a closed polygonal chain. After all, the output is a list of points that we can interpret as the clockwise listing of the vertices of a polygon and any three consecutive points form a right turn, or because of the rounding errors, they almost form a right turn. Moreover, no point in $P$ can be far outside the computed hull. The only problem that can still occur is that, when the three points lie very close together, a turn that is actually a sharp left can be interpreted as a right turn. This might result in a dent in the resulting polygon. A way out of this is to make sure that points in the input that are very close together are considered as being the same point, for example, by rounding. 

### Incremental Convex Hull Algorithm - Time Complexity and Correctness

**Theorem 1.1.** - _The convex hull of a set of n points in the plane can be computed in $O(nlogn)$ time.

_Proof_ - We will prove the correctness of the computation of the upper hull; the lower hull computation can be proved correct using similar arguments. The proof is by induction on the number of points treated. Before the **for**-loop starts, the list $L_{upper}$ contains the points $p_1$ and $p_2$, which trivially form the upper hull of of $\{p_1, p_2\}$. Now suppose that $L_{upper}$ contains the upper hull vertices of $\{p1, \dots, p_{i-1}\}$ and consider the addition of $p_i$. After the execution of the **while**-loop and because of the induction hypothesis, we know that the points in $L_{upper}$ form a chain that only makes the right turns. Moreover, the chain starts at lexicographically smallest point of $\{p_1, \dots, p_i\}$ and ends at lexicographically largest point namely $p_i$. If we can show that all points of $\{p_1, \dots, p_i\}$ that are not in $L_{upper}$ are below the chain, then $L_{upper}$ contains the correct points. By induction we know there is no point above the chain that we had before $p_i$ was added. Since the old chain lies below the new chain, the only possibility for a point to lie above the new chain is if it lies in the vertical slab between $p_{i-1}$ and $p_i$. But this is not possible since such a point would be in between $p_{i-1}$ and $p_i$ in the lexicographical order, with similar argument if the other points have the same $x$-coordinate.

Now for the time bound, we note that sorting the points lexicographically can be done in $O(nlogn)$ time. Now consider the computation of the upper hull. The **for**-loop is executed a linear number of times. The question remains is how often the **while**-loop inside it is executed. For each execution of the **for**-loop the **while**-loop is executed at least once. For any extra execution a point is deleted from the current hull. As each point can be deleted only once during the construction of the upper hull, the total number of extra executions over all **for**-loops is bounded by _n_. Similarly, the computation of the lower hull takes $O(n)$ time. Due to the sorting step, the total time required for computing the convex hull is $O(nlogn)$. $\blacksquare$

**Note** - We have found an efficient and easy to understand solution, but we couldn't have got it just by the original definitions of the problem.

## Degeneracies and Robustness

The development of a geometric algorithm often goes through three phases:

1. *Design* - We try to ignore everything that will clutter our understanding of the geometric concepts we are dealing with. When first trying to design or understand an algorithm, it is often helpful to ignore these degenerate cases.
2. *Adjustment* - We have to adjust the algorithm designed in the first phase to be correct in the presence of degenerate cases. 
3. *Implementation* - Now one needs to think about the primitive operations, like testing whether a point lies to the left, to the right, or on a directed line. 

Another issue that arises in the implementation phase is that the assumption of doing exact arithmetic with real numbers breaks down. One of the solution is to use a package providing exact arithmetic, but it will be slow. The guideline is: If speed is not an issue, then use package with exact arithmetic, but in other cases, it is not so important that the result of the algorithm is precise, just to be generally good.

## Application domains

- Computer Graphics
- Robotics
- Geographic Information Systems
- CAD/CAM
- Other application domains, such as Molecular Biology and Pattern Recognition

# TODO
-----

## Theory
- Convex Hulls
- Convex plane
- How to compute convex hull?
- Clockwise order
- SlowConvexHull Algorithm
- Time Complexity Analysis of SlowConvexHull algorithm
- Degenerate cases of SlowConvexHull
- Floating Point problems, algorithm robustness
- Incremental algorithm
- Degenerate cases of incremental algorithm
- Theorem 1.1. - Time complexity of incremental algorithm for computing convex hull
- Degeneracies and Robustness
- Phases in developing geometric algorithms
- Application domains


## Implementation

- Point class
- orientation method for three given pointss
- Simple polygon from set of vertices _V_
- SlowConvexHull
- RotationAlgorithm
- Graham Scan

## Problems
Exercises (at least 7 of them with solutions)